## Exercício (0,5 ponto na média):

__Exercício:__ o arquivo `sales_data_full.csv` contém o histórico de vendas de 2018 de uma grande rede varejista. O arquivo está dividido da seguinte forma:

`product_id|store_id|[date list]|[quantity list]|[price list]`

Hoje o diretor de vendas gostaria de rever o portfolio das lojas e para isso gostaria de entender:

1. Quais as top5 lojas com maior receita (venda em dinheiro) no histórico da empresa?
2. Quais os top5 produtos produto com maior venda (em quantidade) no histórico da empresa?

__DICA__:
* Estude a função de ordem superior de agrupamento groupby do pacote itertools;
* Observe que esta função agrupa elementos quando a chave está ordenada!

In [1]:
from itertools import groupby

# Carregando e preparando os dados

In [2]:
data_path = '../data/retail_data'

In [3]:
file_2018 = data_path + '/sales_data_2018.csv'
file_full = data_path + '/sales_data_full.csv'

# Criando funções

In [4]:
import re

def ajuste_dados_receita(file):
    
    with open(file) as f:
        lines = f.readlines()
        
    splitted_lines = map(lambda l: l.split('|'), lines)
    
    converted_lines_01 = map(lambda el: [int(el[0]), \
                       int(el[1]), \
                       re.findall('\d{4}-\d{2}-\d{2}', el[2]), \
                       re.findall('\d+\.*\d*', el[3]), \
                       re.findall('\d+\.*\d*', el[4])], splitted_lines)
    
    converted_lines_02 = map(lambda el: el[0:3] + \
                       [[int(i) for i in el[3]]] + \
                       [[float(i) for i in el[4]]], converted_lines_01)
    
    converted_lines = list(converted_lines_02)
    
    #cria a coluna de loja/soma de venda em dinheiro
    receita_lines = sorted(map(lambda line: [line[1]] + [sum(line[-1])], converted_lines), 
                           key = lambda row: row[0])
    
    #agrupa tudo que é da mesma loja em uma única lista
    grouped_list = groupby(receita_lines, key = lambda row: row[0])
    
    #cria uma lista da soma da venda em dinheiro de cada loja
    grouped_list2 = []
    for loja, receita in grouped_list:
        receita_total = sum(item[1] for item in list(receita))
        grouped_list2.append([loja, receita_total])
    
    return grouped_list2
    
    return grouped_list2

In [5]:
import re

def ajuste_dados_produto(file):
    
    with open(file) as f:
        lines = f.readlines()
        
    splitted_lines = map(lambda l: l.split('|'), lines)
    
    converted_lines_01 = map(lambda el: [int(el[0]), \
                       int(el[1]), \
                       re.findall('\d{4}-\d{2}-\d{2}', el[2]), \
                       re.findall('\d+\.*\d*', el[3]), \
                       re.findall('\d+\.*\d*', el[4])], splitted_lines)
    
    converted_lines_02 = map(lambda el: el[0:3] + \
                       [[int(i) for i in el[3]]] + \
                       [[float(i) for i in el[4]]], converted_lines_01)
    
    converted_lines = list(converted_lines_02)
    
    #cria a coluna de produto/soma de venda de produtos em quantidade
    produto_lines = sorted(map(lambda line: [line[0]] + [sum(line[-2])], converted_lines), 
                           key = lambda row: row[0])
    
    #agrupa tudo que é do mesmo produto em uma única lista
    grouped_list = groupby(produto_lines, key = lambda row: row[0])
    
    #cria uma lista da soma da venda em quantidade de cada produto
    grouped_list2 = []
    for produto, venda in grouped_list:
        produto_total = sum(item[1] for item in list(venda))
        grouped_list2.append([produto, produto_total])
    
    return grouped_list2

In [6]:
# testando com a base de 2018

ajuste_dados_receita(file_2018)

[[1, 72432487.13000044],
 [2, 91234989.7600005],
 [3, 37149136.40000004],
 [4, 187991264.69999933],
 [5, 78133596.77999991],
 [6, 23949395.88],
 [7, 37994650.91999993],
 [8, 45071710.85000019],
 [9, 32484778.15999993],
 [10, 33793760.33999994],
 [11, 43125754.23999997],
 [12, 35596357.7499998],
 [13, 21980582.389999963],
 [14, 42843975.169999935],
 [15, 31231201.000000067],
 [16, 32085084.12000005],
 [17, 72383030.70999987],
 [18, 86107519.87000017],
 [19, 148188006.779999],
 [20, 39885542.16000008],
 [21, 99971539.41000007],
 [22, 73890631.34000008],
 [23, 26372872.570000052],
 [24, 25006547.469999906],
 [25, 14388747.539999977],
 [26, 83594696.56999993],
 [27, 24907385.120000042],
 [28, 29700335.809999913],
 [29, 13114325.879999936],
 [30, 39147847.03],
 [31, 10394400.360000022],
 [32, 14901018.06999999],
 [33, 12904501.980000015],
 [34, 8539140.499999998],
 [35, 11052525.090000013],
 [36, 15816487.329999996],
 [37, 69395290.23999996],
 [38, 20852267.710000083],
 [39, 28102871.679999

# Dividir o arquivo principal (file_full) em partes

In [7]:
import os

In [8]:
output_path = 'aula02_data'

In [9]:
# Cria pasta para armazenar partes
try:
    os.mkdir(ouptut_path)
except:
    print('Pasta já existe!')

Pasta já existe!


In [10]:
from support_functions import split_file

In [11]:
files = split_file(file_full, output_path=output_path)

Closing file aula02_data/sales_data_part_64.csv
Finished!


# Encontrando as 5 lojas com maior receita

In [12]:
from multiprocessing import Pool

In [13]:
num_processors = 8

In [14]:
p = Pool(processes = num_processors)

In [15]:
import glob

files = glob.glob(output_path + '/sales_data_part*.csv')

In [16]:
# unindo as listas de cada arquivo
ajuste_per_file = p.map(ajuste_dados_receita, files)

In [17]:
ajuste_per_file[0:2]

[[[1, 16232925.049999991],
  [2, 21017981.660000034],
  [3, 10254315.699999988],
  [4, 43155906.470000036],
  [5, 18265189.02],
  [6, 5321021.770000004],
  [7, 8834039.469999995],
  [8, 11669168.859999998],
  [9, 8105545.699999991],
  [10, 7696273.489999999],
  [11, 11357384.60999998],
  [12, 8059397.49999999],
  [13, 5062181.720000005],
  [14, 9433276.78],
  [15, 6974271.290000006],
  [16, 7414800.709999992],
  [17, 19059751.67999997],
  [18, 17975367.850000024],
  [19, 37442735.94999999],
  [20, 10297370.540000018],
  [21, 25209927.350000024],
  [22, 16921707.16],
  [23, 6578691.349999996],
  [24, 5492166.120000002],
  [25, 2949096.6699999967],
  [26, 18005440.189999998],
  [27, 5543176.349999993],
  [28, 7397764.029999998],
  [29, 2755693.880000003],
  [30, 9053218.889999993],
  [31, 2132576.2399999984],
  [32, 3346522.2900000005],
  [33, 2154744.589999999],
  [34, 1772970.8799999997],
  [35, 2460645.110000001],
  [36, 3851747.829999996],
  [37, 15152586.709999992],
  [38, 4365934.6

In [18]:
total_receita = []

for lst in ajuste_per_file:
    total_receita = total_receita + lst

In [19]:
total_receita

[[1, 16232925.049999991],
 [2, 21017981.660000034],
 [3, 10254315.699999988],
 [4, 43155906.470000036],
 [5, 18265189.02],
 [6, 5321021.770000004],
 [7, 8834039.469999995],
 [8, 11669168.859999998],
 [9, 8105545.699999991],
 [10, 7696273.489999999],
 [11, 11357384.60999998],
 [12, 8059397.49999999],
 [13, 5062181.720000005],
 [14, 9433276.78],
 [15, 6974271.290000006],
 [16, 7414800.709999992],
 [17, 19059751.67999997],
 [18, 17975367.850000024],
 [19, 37442735.94999999],
 [20, 10297370.540000018],
 [21, 25209927.350000024],
 [22, 16921707.16],
 [23, 6578691.349999996],
 [24, 5492166.120000002],
 [25, 2949096.6699999967],
 [26, 18005440.189999998],
 [27, 5543176.349999993],
 [28, 7397764.029999998],
 [29, 2755693.880000003],
 [30, 9053218.889999993],
 [31, 2132576.2399999984],
 [32, 3346522.2900000005],
 [33, 2154744.589999999],
 [34, 1772970.8799999997],
 [35, 2460645.110000001],
 [36, 3851747.829999996],
 [37, 15152586.709999992],
 [38, 4365934.66],
 [39, 6160462.829999991],
 [40, 50

In [20]:
receita_grouped = groupby(sorted(total_receita, key = lambda row: row[0]), key = lambda row: row[0])

In [21]:
receita_grouped2 = []

for loja, receitas in receita_grouped:
    total_receita = sum(item[1] for item in list(receitas))
    receita_grouped2.append([loja, total_receita])

In [22]:
receita_grouped2

[[1, 920401804.7000003],
 [2, 1158056334.5300002],
 [3, 494637852.32000005],
 [4, 2545458074.9900002],
 [5, 1003174668.9899997],
 [6, 307261078.93],
 [7, 489655899.15999997],
 [8, 595794589.5499998],
 [9, 429712035.56000006],
 [10, 430185358.04999983],
 [11, 562090226.8899999],
 [12, 453873053.0999997],
 [13, 281944422.14],
 [14, 550099360.2399999],
 [15, 396265675.5099999],
 [16, 433206774.22],
 [17, 976941427.6800002],
 [18, 1081100615.5500002],
 [19, 1984132672.7800007],
 [20, 517243320.0199999],
 [21, 1399232241.8],
 [22, 938954858.9399998],
 [23, 356519363.2999999],
 [24, 311616691.0300001],
 [25, 182029911.26999995],
 [26, 1056860475.55],
 [27, 316906406.34],
 [28, 402076605.13],
 [29, 164068210.8200001],
 [30, 497605064.9999998],
 [31, 131097516.73000005],
 [32, 185930631.11000007],
 [33, 171864103.57999992],
 [34, 107315900.55999994],
 [35, 141454002.95000005],
 [36, 199613238.77999997],
 [37, 903615501.67],
 [38, 257961751.7499999],
 [39, 374315094.14000005],
 [40, 286797481.8

In [23]:
receita_top5 = sorted(receita_grouped2, key = lambda row: row[-1], reverse = True)[0:5]

In [24]:
receita_top5

[[4, 2545458074.9900002],
 [19, 1984132672.7800007],
 [21, 1399232241.8],
 [2, 1158056334.5300002],
 [18, 1081100615.5500002]]

# Encontrando os 5 produtos com maior venda em quantidade

In [25]:
# unindo as listas de cada arquivo
ajuste_per_file2 = p.map(ajuste_dados_produto, files)

In [26]:
total_produto = []

for lst in ajuste_per_file2:
    total_produto = total_produto + lst

In [27]:
total_produto

[[1, 11],
 [2, 5],
 [3, 5],
 [4, 2777],
 [5, 4271],
 [6, 14761],
 [7, 20067],
 [8, 31282],
 [9, 55784],
 [10, 23],
 [11, 4233],
 [12, 532],
 [13, 944],
 [14, 869],
 [15, 3043],
 [16, 3],
 [17, 2],
 [18, 21],
 [19, 3],
 [20, 5],
 [21, 3],
 [22, 263],
 [23, 164],
 [24, 981],
 [25, 969],
 [26, 72],
 [27, 96],
 [28, 1077],
 [29, 1],
 [30, 8],
 [31, 3800],
 [32, 3727],
 [33, 11449],
 [34, 6519],
 [35, 10683],
 [36, 39315],
 [37, 11027],
 [38, 6128],
 [39, 10150],
 [40, 4147],
 [41, 2055],
 [42, 699],
 [43, 3122],
 [44, 8814],
 [45, 2692],
 [46, 2],
 [47, 4241],
 [48, 4835],
 [49, 9215],
 [50, 6590],
 [51, 2712],
 [52, 5107],
 [53, 5221],
 [54, 12718],
 [55, 13585],
 [56, 18323],
 [57, 14952],
 [58, 8765],
 [59, 10366],
 [60, 1],
 [61, 2653],
 [62, 4041],
 [63, 1574],
 [64, 2536],
 [65, 5185],
 [66, 9987],
 [67, 12],
 [68, 3150],
 [69, 6528],
 [70, 6381],
 [71, 5546],
 [72, 24993],
 [73, 10941],
 [74, 1708],
 [75, 2060],
 [76, 61826],
 [77, 37201],
 [78, 75621],
 [79, 20],
 [80, 30],
 [81, 7

In [28]:
produto_grouped = groupby(sorted(total_produto, key = lambda row: row[0]), key = lambda row: row[0])

In [29]:
produto_grouped2 = []

for produto, venda in produto_grouped:
    total_produto = sum(item[1] for item in list(venda))
    produto_grouped2.append([produto, total_produto])

In [30]:
produto_grouped2

[[1, 81],
 [2, 133],
 [3, 35],
 [4, 33905],
 [5, 59768],
 [6, 176304],
 [7, 234783],
 [8, 432036],
 [9, 805648],
 [10, 23315],
 [11, 32359],
 [12, 13816],
 [13, 6193],
 [14, 6481],
 [15, 20150],
 [16, 13551],
 [17, 36],
 [18, 8192],
 [19, 37],
 [20, 80],
 [21, 21620],
 [22, 34687],
 [23, 56813],
 [24, 46979],
 [25, 76159],
 [26, 194647],
 [27, 73560],
 [28, 41318],
 [29, 53058],
 [30, 13589],
 [31, 24590],
 [32, 28507],
 [33, 75685],
 [34, 104945],
 [35, 96218],
 [36, 300059],
 [37, 95748],
 [38, 105795],
 [39, 109922],
 [40, 41871],
 [41, 48450],
 [42, 34754],
 [43, 93233],
 [44, 165335],
 [45, 110468],
 [46, 69741],
 [47, 80682],
 [48, 110264],
 [49, 62870],
 [50, 42907],
 [51, 19380],
 [52, 53757],
 [53, 60824],
 [54, 91624],
 [55, 98416],
 [56, 158487],
 [57, 162003],
 [58, 63224],
 [59, 123894],
 [60, 42957],
 [61, 50083],
 [62, 198082],
 [63, 118964],
 [64, 29374],
 [65, 48259],
 [66, 468709],
 [67, 261130],
 [68, 519660],
 [69, 46911],
 [70, 50954],
 [71, 68402],
 [72, 270998],


In [31]:
produto_top5 = sorted(produto_grouped2, key = lambda row: row[-1], reverse = True)[0:5]

In [32]:
produto_top5

[[643, 986581], [11096, 806005], [9, 805648], [2322, 677642], [1908, 667034]]